# Text Classification

In dieser Aufgabe werden Sie eine Text Classification Pipeline bauen, die Partei gegeben einen Text vorhersagt. 
Statt der Parlamentsdebatten koennen Sie auch gerne einen anderen Text Datensatz nehmen, wenn Sie einen guten finden.
Stellen Sie aber sicher, dass Ihre Kollegen Zugriff auf die Daten haben fuer die Korrektur. 

In [1]:
import os, gzip
import pandas as pd
import numpy as np
import urllib.request

import warnings
warnings.filterwarnings('ignore')

DATADIR = "data"

if not os.path.exists(DATADIR): 
    os.mkdir(DATADIR)

file_name = os.path.join(DATADIR, 'bundestags_parlamentsprotokolle.csv.gzip')
if not os.path.exists(file_name):
    url_data = 'https://www.dropbox.com/s/1nlbfehnrwwa2zj/bundestags_parlamentsprotokolle.csv.gzip?dl=1'
    urllib.request.urlretrieve(url_data, file_name)

df = pd.read_csv(gzip.open(file_name), index_col=0).sample(frac=1)

Ein Auszug der Parlamentsdebatten

In [2]:
df[:4]

,sitzung,wahlperiode,sprecher,text,partei
15304,176,17,Johannes Vogel,Herr Präsident! Liebe Kolleginnen und Kollegen...,fdp
10444,124,17,Georg Schirmbeck,"Meine Damen und Herren, auch wenn wir eine frö...",cducsu
31206,87,18,Franz-Josef Holzenkamp,"zeigt die Tatsache, dass Frau Aigner bereits i...",cducsu
42263,228,18,Dr. Johannes Fechner,Wir haben noch vor der letzten Sommerpause mit...,spd


Splitten Sie die Daten in Train (80%) und Test (20%), dafuer koennen sie die sklearn train_test_split function benutzen. 

Dann trainieren Sie eine Pipeline mit einem geeigneten Vectorizer und einem sklearn Modell Ihrer Wahl. 

Vergleichen Sie die Precision/Recall/F1 und Accuracy auf dem Train und Test set. 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report


train_data, test_data, train_labels, test_labels = train_test_split(df['text'], df['partei'], test_size=0.3)

ncc_clf = Pipeline([('vect', TfidfVectorizer(max_features=int(1e8))),
                            ('clf', NearestCentroid())]).fit(train_data, train_labels)

logreg_clf = Pipeline([('vect', TfidfVectorizer(max_features=int(1e8))),
                        ('clf', SGDClassifier())]).fit(train_data, train_labels)

In [4]:
##Evaluating NCC on Training Data

ncc_predictions = ncc_clf.predict(train_data)
print(classification_report(ncc_predictions, train_labels))

## Evaluating NCC on Test Data

ncc_predictions_test = ncc_clf.predict(test_data)
print(classification_report(ncc_predictions_test, test_labels))



              precision    recall  f1-score   support

      cducsu       0.49      0.65      0.56      8440
         fdp       0.44      0.22      0.30      4579
      gruene       0.46      0.35      0.40      5850
       linke       0.61      0.37      0.46      7005
         spd       0.28      0.50      0.36      4701

    accuracy                           0.44     30575
   macro avg       0.46      0.42      0.42     30575
weighted avg       0.47      0.44      0.44     30575

              precision    recall  f1-score   support

      cducsu       0.48      0.62      0.54      3758
         fdp       0.35      0.18      0.23      2032
      gruene       0.39      0.31      0.34      2302
       linke       0.57      0.35      0.44      2936
         spd       0.27      0.46      0.34      2076

    accuracy                           0.41     13104
   macro avg       0.41      0.39      0.38     13104
weighted avg       0.43      0.41      0.40     13104



In [5]:
## Evaluating Logistic Regression on Training Data

logreg_predictions = logreg_clf.predict(train_data)
print(classification_report(logreg_predictions, train_labels))

## Evaluating Logistic Regression on Training Data

logreg_predictions_test = logreg_clf.predict(test_data)
print(classification_report(logreg_predictions_test, test_labels))

              precision    recall  f1-score   support

      cducsu       0.96      0.69      0.80     15729
         fdp       0.33      0.99      0.50       790
      gruene       0.73      0.96      0.83      3359
       linke       0.90      0.87      0.89      4378
         spd       0.66      0.87      0.75      6319

    accuracy                           0.79     30575
   macro avg       0.72      0.88      0.75     30575
weighted avg       0.85      0.79      0.80     30575

              precision    recall  f1-score   support

      cducsu       0.90      0.57      0.69      7681
         fdp       0.05      0.58      0.10        95
      gruene       0.29      0.60      0.39       901
       linke       0.66      0.61      0.63      1995
         spd       0.40      0.59      0.47      2432

    accuracy                           0.58     13104
   macro avg       0.46      0.59      0.46     13104
weighted avg       0.72      0.58      0.62     13104

